<a href="https://colab.research.google.com/github/szucshey/osm-data-exploration/blob/main/census/census_data_in_hungary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install folium osm2geojson geopandas


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.8 MB/s 
     |████████████████████████████████| 6.3 MB 30.0 MB/s 
     |████████████████████████████████| 16.7 MB 44.5 MB/s 
  Created wheel for osm2geojson: filename=osm2geojson-0.2.0-py3-none-any.whl size=13345 sha256=5c6a80ea1daf4c097fe196a318d220d3d6b061efd9c4ff1cd502c884b773c74b
  Stored in directory: /root/.cache/pip/wheels/de/69/12/fe31a7e75bd52382aabbeac95e0137776e991f0a6f9295b170
Successfully built osm2geojson


In [2]:
import requests
import json
#Magyarország településhatárainak, megyehatárainak, a főváros kerületeinek határainak lekérése az OSM API-n keresztül
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
    [out:json];
    rel["wikipedia"~"^hu:"]
    [admin_level=8]
    [type=boundary]
    [name!~"Balatonakarattya"]
    [boundary=administrative];
    out geom;

    rel["wikipedia"~"^hu:"]
    [admin_level=9]
    [type=boundary]
    [boundary=administrative];
  	out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("towns.json", "w")
json.dump(response.json(), output)
output.close()

######################################################################################################################

overpass_query = """
    [out:json];
    rel["ISO3166-2"~"^HU"]
    [admin_level=6]
    [type=boundary]
    [boundary=administrative];
    out geom;
    """

response = requests.get(overpass_url, params={'data': overpass_query})

#Az API válasz lementése egy json fileba
output = open("states.json", "w")
json.dump(response.json(), output)
output.close()

In [8]:
import folium
import osm2geojson

#A térkép létrehozása foliumban, kezdeti koordináták, zoom-lehetőségek, térképvászon megadása, az attribútum kötelező külső vászon használata esetén
m = folium.Map(location=[47.161, 19.505], zoom_start=7, min_zoom=7, maxBounds=[[40.000, 10.000], [60.000, 30.000]], tiles='https://{s}.basemaps.cartocdn.com/dark_nolabels/{z}/{x}/{y}{r}.png', attr="-", prefer_canvas=True)

style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#fff", 'weight': 2}
data = json.load(open("states.json", "r"))
geojson = osm2geojson.json2geojson(data)

g = folium.GeoJson(geojson, name='geojson', style_function=style_function, smooth_factor=2.0, embed=False).add_to(m)

style_function = lambda x: {'fillColor': '#000000', 'fillOpacity': 0.0, 'stroke': True, 'color': "#fff", 'weight': 0.5}
#A korábban kapott json fájl geojson formátumba való átalakítása
data = json.load(open("towns.json", "r"))
geojson = osm2geojson.json2geojson(data)

g = folium.GeoJson(geojson, name='geojson', style_function=style_function, smooth_factor=2.0, embed=False).add_to(m)

fgs = [folium.FeatureGroup(name="Németek", overlay=True, control=True, show=True).add_to(m), 
       folium.FeatureGroup(name="Szlovákok", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Románok", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Horvátok", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Szerbek", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Ukránok", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Szlovének", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Romák", overlay=True, control=True, show=True).add_to(m),
       folium.FeatureGroup(name="Egyéb nemzetiségek", overlay=True, control=True, show=True).add_to(m)]


/usr/local/lib/python3.7/dist-packages/osm2geojson/main.py:514: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in merged_line:


In [4]:
from shapely import geometry
import random

In [5]:
def get_random_point_in_polygon(poly):
  
  minx, miny, maxx, maxy = poly.bounds
  while True:
       p = geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
       if poly.contains(p):
          return p

In [6]:
import pandas as pd

df = pd.read_csv("data_filtered.csv", delimiter=";")

nemzetisegek = ["Német", "Szlovák", "Román", "Horvát", "Szerb", "Ukrán", "Szlovén", "Roma", "Egyéb"]
color_codes = ["#FFF712", "#09EC8E", "#D57C1E", "#D57C1E", "#DC1111", "#0BF7FF", "#FF74E8", "#A2059D", "#1E9111"]
ev = 2011

In [11]:
for n in range(8):

  nemzetiseg = nemzetisegek[n]
  df2 = df[df["Év"] == ev][df["Kategória"] == "A-N-H-K"][["Település", nemzetiseg]]
  fill = color_codes[n]

  for i in range(len(geojson['features'])):
    poly = geometry.Polygon([[coord[0], coord[1]] for coord in geojson['features'][i]['geometry']['coordinates'][0][0]]) 
  
    #adott megyén belül minden x (representation) főre generálódik egy pont
    #representation = 10
    try:
    #if geojson['features'][i]['properties']['tags']['name'] in df['Település'].values:
      popup = geojson['features'][i]['properties']['popup'] = "<b>{0}</b>".format(geojson['features'][i]['properties']['tags']['name']) + ": {0}".format(int(df2[df2["Település"] == geojson['features'][i]['properties']['tags']['name']][nemzetiseg]))
    
      for j in range(int(df2[df2["Település"] == geojson['features'][i]['properties']['tags']['name']][nemzetiseg])): #// representation):
        point = get_random_point_in_polygon(poly)
        folium.Circle(location=[point.y, point.x], radius=10, fill_color=fill, fill_opacity=1.0, opacity=0.0).add_to(fgs[n])
    except:
      print(geojson['features'][i]['properties']['tags']['name'], df2[df2["Település"] == geojson['features'][i]['properties']['tags']['name']][nemzetiseg])
  
  folium.features.GeoJsonTooltip(['popup'], labels=False).add_to(g)
  g.add_to(fgs[n])

folium.LayerControl(position='bottomleft').add_to(m)

m.save(outfile="map.html")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


Óbánya Series([], Name: Német, dtype: int64)
XXIII. kerület Series([], Name: Német, dtype: int64)
Óbánya Series([], Name: Szlovák, dtype: int64)
XXIII. kerület Series([], Name: Szlovák, dtype: int64)
Óbánya Series([], Name: Román, dtype: int64)
XXIII. kerület Series([], Name: Román, dtype: int64)
Óbánya Series([], Name: Horvát, dtype: int64)
XXIII. kerület Series([], Name: Horvát, dtype: int64)
Óbánya Series([], Name: Szerb, dtype: int64)
XXIII. kerület Series([], Name: Szerb, dtype: int64)
Óbánya Series([], Name: Ukrán, dtype: int64)
XXIII. kerület Series([], Name: Ukrán, dtype: int64)
Óbánya Series([], Name: Szlovén, dtype: int64)
XXIII. kerület Series([], Name: Szlovén, dtype: int64)
Óbánya Series([], Name: Roma, dtype: int64)
XXIII. kerület Series([], Name: Roma, dtype: int64)


In [12]:
from google.colab import files
files.download('map.html') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>